# Experian Gross to Net Income Task 1

### You should submit this notebook, the salary_holdout.csv with the populated net to gross incomes, and a word/pdf doc for task 2. Submit the csv accoridng to the format it is received, with the annual_gross_income column filled with appropriate values.

----------
Written by Lucas Murphy and Emrah Selcuk 2022

## Purpose of this task

By the end of this task you should:
    
    - have an in depth understanding of tax system used in the UK
    - be able to reverse net back into gross income
    - understand some of the challenges faced when inferring gross income from net

### Setup

Read in CSV of incomes...

In [20]:
import pandas as pd

df = pd.read_csv('salary_holdout_23.csv')
df

,location,monthly_net_income,annual_gross_income
0,rou,4337.10,0.0
1,sco,1678.07,0.0
2,rou,2096.05,0.0
3,rou,2816.73,0.0
4,sco,2645.94,0.0
...,...,...,...
4995,rou,1312.26,0.0
4996,rou,3508.59,0.0
4997,rou,4275.59,0.0
4998,rou,3141.03,0.0


### Data Inspection

#### Identifying if there are any null or incorrect values in the dataset 

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   location             5000 non-null   object 
 1   monthly_net_income   5000 non-null   float64
 2   annual_gross_income  5000 non-null   float64
dtypes: float64(2), object(1)
memory usage: 117.3+ KB


In [22]:
df.sort_values(by="monthly_net_income",ascending=True)

,location,monthly_net_income,annual_gross_income
1803,rou,-3366.08,0.0
2260,rou,-2752.40,0.0
4099,sco,-2058.30,0.0
39,sco,-1250.91,0.0
3389,rou,-1162.49,0.0
...,...,...,...
718,rou,6948.05,0.0
2424,rou,7251.11,0.0
2627,rou,7262.32,0.0
4599,sco,7520.27,0.0


#### Observation: This dataset has negative values, which will not be useful in calculating annual gross income. Hence, removing the negative values.

#### Removing negative values

In [23]:
if (df['monthly_net_income'].values < 0).any():
    df = df[(df['monthly_net_income']>0)]


(df['monthly_net_income'].values < 0).any()

False

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4995 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   location             4995 non-null   object 
 1   monthly_net_income   4995 non-null   float64
 2   annual_gross_income  4995 non-null   float64
dtypes: float64(2), object(1)
memory usage: 156.1+ KB


#### Observation: After removing the negative values, the dataset now has 4995 values

In [25]:
df.head(20)

,location,monthly_net_income,annual_gross_income
0,rou,4337.10,0.0
1,sco,1678.07,0.0
2,rou,2096.05,0.0
3,rou,2816.73,0.0
4,sco,2645.94,0.0
5,sco,1688.47,0.0
6,rou,1938.92,0.0
7,rou,1199.49,0.0
8,rou,2953.19,0.0
9,rou,2743.55,0.0


In [26]:
df.describe()

,monthly_net_income,annual_gross_income
count,4995.000000,4995.0
mean,2817.517598,0.0
std,1060.252055,0.0
min,544.440000,0.0
25%,2020.445000,0.0
50%,2699.010000,0.0
75%,3456.470000,0.0
max,7783.880000,0.0


### Functionality

Write a program that converts monthly net income to annual gross income. You can assume that the monthly income is constant month to month. It is *strongly* advised to first write functionality to convert gross to net and confirm this with the provided test set. Your grade for this section will be based solely on your calculations for the holdout set.

For transparency, write your algorithm according to the tax brackets and rates communicated in the presentation slides.

#### The tax brackets and rates for UK and Scotland, including National Insurance

In [27]:
    # Rest of UK Income Tax brackets and rates
uk_tax_ni_table = [
    (0, 12570, 0),              # Personal Allowance
    (12571, 50270, 20),         # Basic Rate
    (50271, 125140, 40),        # Higher Rate
    (125141, float('inf'), 45), # Additional Rate

    # Rest of UK National Insurance brackets and rates
    (0, 11904, 0),              # NI No payment
    (11905, 50270, 13.25),      # NI Lower
    (50271, float('inf'), 3.25) # NI Higher
]
 
 
    # Scotland Income Tax brackets and rates
scotland_tax_ni_table = [
    (0, 12570, 0),              # Personal Allowance
    (12571, 14732, 19),         # Starter Rate
    (14733, 25688, 20),         # Basic Rate
    (25689, 43662, 21),         # Intermediate Rate
    (43663, 125140, 42),        # Higher Rate
    (125141, float('inf'), 47), # Top Rate

    # Scotland National Insurance brackets and rates
    (0, 11904, 0),              # NI No payment
    (11905, 50270, 13.25),      # NI Lower
    (50271, float('inf'), 3.25) # NI Higher
]

#### Calculate the net income from the gross income based on the tax system
    Arguments:
    gross_income (float): The gross income of the individual.
    tax_table (list): A list of tuples representing the tax brackets and rates.
    Returns:
    float: The estimated net income.

In [28]:
def gross_to_net(gross_income, tax_table):
    net_income = gross_income
    tax_paid = 0
    # Calculate the net income by subtracting the tax for each bracket
    for lower_bound, upper_bound, rate in tax_table:
        if gross_income > lower_bound:
            taxable_income = min(gross_income, upper_bound) - lower_bound
            tax_paid += taxable_income * (rate / 100)
    net_income -= tax_paid  #To get net income subtract the tax from the gross income  
    #print("Net Income",net_income)
    return net_income       # Returns the estimated net income
 

#### Reverse calculation of the gross income from the net income by iteratively applying the gross_to_net function.
    Arguments:
    net_income (float): The net income of the individual.
    tax_table (list): A list of tuples representing the tax brackets and rates.
    Returns:
    float: The estimated gross income.

In [29]:
def net_to_gross(net_income, tax_table):
 
    lower_bound = 0
    upper_bound = net_income * 2  # Set an upper bound that is definitely above the gross income
    tolerance = 1                 # Set a tolerance level to determine how close the net calculation should be
    # Perform a binary search to efficiently find the gross income estimate
    while lower_bound <= upper_bound:
        mid_point = (lower_bound + upper_bound) / 2
        net_estimate = gross_to_net(mid_point, tax_table)
        if abs(net_estimate - net_income) < tolerance:
            return mid_point
        elif net_estimate < net_income:
            lower_bound = mid_point + 1
        else:
            upper_bound = mid_point - 1
    return lower_bound            # If the loop exits, the lower_bound is the best estimate

#### Calculate the estimated gross income for each monthly net income in dataframe

In [30]:

for index, row in df.iterrows():       #Looping through each entry of monthly net income                      
    # Finding the approx annual net income by multiplying the monthly net income by 12
    net_income_annual = row['monthly_net_income'] * 12
    # Selecting the correct tax table based on the location (rou or sco)
    location = row['location'] 
    tax_table = uk_tax_ni_table if location == 'rou' else scotland_tax_ni_table
    # Calculating the estimated gross income
    annual_gross_income = net_to_gross(net_income_annual, tax_table)      
    # Estimated gross income
    df.loc[index, 'annual_gross_income'] = annual_gross_income       
 

In [31]:
df.head(30)

,location,monthly_net_income,annual_gross_income
0,rou,4337.10,75641.340967
1,sco,1678.07,24006.097832
2,rou,2096.05,31551.538281
3,rou,2816.73,44508.226699
4,sco,2645.94,41642.859414
5,sco,1688.47,24191.978740
6,rou,1938.92,28726.167549
7,rou,1199.49,15434.135000
8,rou,2953.19,46960.804692
9,rou,2743.55,43191.130029


In [32]:
df.to_csv('salary_holdout_23.csv', index=False) #exporting the salary_holdout.csv file for client review